## seq2seq Slack Bot

![](https://cdn-images-1.medium.com/max/2560/1*1I2tTjCkMHlQ-r73eRn4ZQ.png)

Dictionary{ id: word } or List of words [["I", "am" ,"Akira"], ["He", "is" ,"Vivec"]] ???

In [10]:
import json
import os
import shutil

import numpy as np
from keras.layers import Input, Embedding, LSTM, TimeDistributed, Dense
from keras.models import Model

In [12]:
def seq2seq_model_generator(
        input_dict_size, 
        output_dict_size,
        input_length=20,
        output_length=20):
    
    #  define length of encoder/decoder
    encoder_input = Input(shape=(input_length, ))
    decoder_input = Input(shape=(output_length, ))
    
    #  encoder
    encoder_input2embedding = Embedding(input_dim=imput_dict_size,
                        output_dim=64,
                        input_length=input_length,
                        mask_zero=True)(encoder_input)
    embedding2encoder = LSTM(units=64, return_sequences=True)(encoder_input2embedding)
    
    # decoder
    decoder_input2embedding = Embedding(input_dim=output_dict_size,
                        output_dim=64,
                        input_length=output_length,
                        mask_zero=True)(decoder_input)
    embedding2decoder = LSTM(units=64, return_sequences=True)(decoder_input2embedding)
    decoder2output = TimeDistributed(Dense(units=output_dict_size, activation="softmax"))(embedding2decoder)
    
    model = Model(inputs=[encoder_input, decoder_input], outputs=[decoder2output])
    model.compile(optimizer="adam", loss="categorical_crossentropy")
    
    return model

In [13]:
from keras.layers import Activation, dot, concatenate

In [ ]:
encoder2attention = dot([embedding2decoder, embedding2encoder], axes=[2, 2])
attention2softmax = Activation("softmax", name="attention")(encoder2attention)

context_vector = dot([attention2softmax, embedding2encoder], axes=[2, 1])

context_vector_decoder = concatenate([context_vector, embedding2decoder])

decoder2tanh = Dense(units=64, activation="tanh")
tanh2context = TimeDistributed(decoder2tanh)(context_vector_decoder)
context2softmax = Dense(units=output_dict_size, activation="softmax")
softmax2output = TimeDistributed(context2softmax)(tanh2context)